# Wrapping Up Offline RL as part of AutoMLPipeline Workflow

- Paulito Palmes
- IBM Research Europe
- Dublin Research Lab

## Preliminaries


### Online RL vs Offline RL

### Online RL

<img src="rl.png" alt="online RL" width="800" />

- maximize return (accumulation of sum of discounted current and future rewards)
- each observation is part of a sequence of <s,a,r> trajectory
- each action influences future observations and accumulated rewards
- unlike in typical ML problem where the objective is to make one time prediction of the action to take, RL makes a series of predictions dynamically as it receives observations and optimizes accumulation of corresponding rewards

ref: https://rail.eecs.berkeley.edu/deeprlcourse/

### Offline RL

<img src="offlinerl.png" alt="offline RL" width="1000" />

- in online RL, new data is collected for policy update
- in off-policy RL, some old data are retained together with new data for policy update
- in offline RL, all data is collected in advanced to train an RL agent for optimal policy by sampling episodes

ref: https://rail.eecs.berkeley.edu/deeprlcourse/

### Why Offline RL

- Cost: it can be too expensive to interact repeatedly and explore certain environment such as workload or resource management in the cloud but cheaper to collect logs and statistics for offline RL learning
- Risk: it can be risky to train an agent in autonomous driving and robotic operations
- Technological advancement in batch learning with deep learning architectures that can scale-up effectively for large datasets

### Major Objective

Given a dataset containing <state, action, reward> trajectories, create an AutoMLPipeline wrapper function for offline RL to make it trivial to search for the best data processing pipeline for offline RL application.

### Load packages 

In [87]:
using Distributed

nprocs() == 1 && addprocs() 

@everywhere begin
   using AutoOfflineRL
   using AutoMLPipeline
   using Parquet
   using DataFrames
end

### Load Preprocessing Elements

In [102]:
@everywhere begin
   #### Scaler
   rb = SKPreprocessor("RobustScaler");
   pt = SKPreprocessor("PowerTransformer");
   norm = SKPreprocessor("Normalizer");
   mx = SKPreprocessor("MinMaxScaler");
   std = SKPreprocessor("StandardScaler")
   ##### Column selector
   catf = CatFeatureSelector();
   numf = NumFeatureSelector();
   #### feature extractors
   pca = SKPreprocessor("PCA");
   fa = SKPreprocessor("FactorAnalysis");
   ica = SKPreprocessor("FastICA");
   noop = Identity(Dict(:name => "Noop"));
   #### ML/RL agents
   rf  = RandomForest()
   tree = PrunedTree()
   dqn = DiscreteRLOffline("DQN")
   sac = DiscreteRLOffline("DiscreteSAC")
end

### Load Offline Dataset

In [89]:
path = pkgdir(AutoOfflineRL)
dataset = "$path/data/smalldata.parquet"
df = Parquet.read_parquet(dataset) |> DataFrame |> dropmissing
first(df,10)

Row,day,hour,minute,dow,sensor1,sensor2,sensor3,action,reward
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,0,0,2,7,110,25,10,0.838122
2,1,0,0,2,444,50,56,10,0.639387
3,1,0,0,2,9,138,61,100,0.416196
4,1,0,0,2,365,51,26,10,0.384344
5,1,0,0,2,312,129,42,10,0.37681
6,1,0,0,2,295,141,58,10,0.641975
7,1,0,1,2,92,18,46,10,0.225288
8,1,0,1,2,77,23,4,50,0.335901
9,1,0,1,2,131,61,13,100,0.993489


### Convert dataframe to MDP dataset

In [90]:
srow,_ = size(df)

reward = df[:,["reward"]] |> deepcopy |> DataFrame
action = df[:,["action"]] |> deepcopy |> DataFrame
_terminals = zeros(Int,srow)
_terminals[collect(100:1000:9000)] .= 1
_terminals[end] = 1
terminaldf = DataFrame(terminal=_terminals)

observation = df[:, ["day", "hour", "minute", "dow", "sensor1", "sensor2", "sensor3"]]
action_reward_terminal = DataFrame[action, reward, terminaldf];

### Recalling the AutoML Pipeline Workflow

In [105]:
mypipeline = numf |> std
tr = fit_transform!(mypipeline,observation)
first(tr,3)

Row,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.396769,-1.38433,-1.69854,-0.396769,-1.68999,0.172168,-0.878338
2,-0.396769,-1.38433,-1.69854,-0.396769,1.34805,-0.86808,0.200461
3,-0.396769,-1.38433,-1.69854,-0.396769,-1.67609,0.657617,0.374461


In [106]:
mypipeline = numf |> std |> pca
tr = fit_transform!(mypipeline,observation)
first(tr,3)

Row,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.162033,0.283571,0.320088,-0.760166,2.38647,1.51367,-4.96579e-17
2,0.158823,-1.50436,1.51579,-0.755993,-0.22961,1.58652,-3.16875e-17
3,0.163828,0.151378,0.754364,0.502795,2.33643,1.47229,-4.6244e-17


In [107]:
mypipeline = numf |> std |> pca |> rf
perf = crossvalidate(mypipeline,observation,action.action)

fold: 1, 33.800000000000004
fold: 2, 31.2
fold: 3, 31.2
fold: 4, 31.5
fold: 5, 33.5
fold: 6, 33.033033033033036
fold: 7, 34.0
fold: 8, 32.300000000000004
fold: 9, 34.2
fold: 10, 34.9
errors: 0


(mean = 32.9633033033033, std = 1.3394702396340163, folds = 10, errors = 0)

### Check crossvalidation performance of an NFQ agent

In [108]:
nfq = DiscreteRLOffline("NFQ")
pipe = (numf |> mx |> pca) |> nfq
tderror=crossvalidateRL(pipe,observation,action_reward_terminal)

Epoch 3/3: 100%|█████████████| 218/218 [00:00<00:00, 767.65it/s, loss=4.62e+11]


2023-06-22 04:51:12 [debug    ] RoundIterator is selected.
2023-06-22 04:51:12 [info     ] Directory is created at d3rlpy_logs/NFQ_20230622045112
2023-06-22 04:51:12 [debug    ] Fitting scaler...              scaler=min_max
2023-06-22 04:51:13 [debug    ] Building models...
2023-06-22 04:51:13 [debug    ] Models have been built.
2023-06-22 04:51:13 [info     ] Parameters are saved to d3rlpy_logs/NFQ_20230622045112/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': {'type': 'min_max', 'params': {'maximum': array

4.7952334405060536e23

In [109]:
tderror

4.7952334405060536e23

The TD error is the difference between the agent's current estimate and target value

### Check validation performance of DQN agent

In [110]:
dqn = DiscreteRLOffline("DQN")
pipe = (numf |> std |> ica) |> dqn
tderror=crossvalidateRL(pipe,observation,action_reward_terminal)

/Users/ppalmes/phome/julia/AutoMLPipeline.jl/AutoOfflineRL/examples/.CondaPkg/env/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/Users/ppalmes/phome/julia/AutoMLPipeline.jl/AutoOfflineRL/examples/.CondaPkg/env/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
Epoch 3/3: 100%|███████████████| 246/246 [00:00<00:00, 866.36it/s, loss=0.0435]


2023-06-22 04:53:17 [debug    ] RoundIterator is selected.
2023-06-22 04:53:17 [info     ] Directory is created at d3rlpy_logs/DQN_20230622045317
2023-06-22 04:53:17 [debug    ] Fitting scaler...              scaler=min_max
2023-06-22 04:53:17 [debug    ] Building models...
2023-06-22 04:53:17 [debug    ] Models have been built.
2023-06-22 04:53:17 [info     ] Parameters are saved to d3rlpy_logs/DQN_20230622045317/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': {'type': 'min_max', 'params': {'maximum': array

1.2257244226676562

In [111]:
tderror

1.2257244226676562

The TD error is the difference between the agent's current estimate and target value

### Find optimal OfflineRL pipeline in parallel

In [29]:
function pipelinesearch()
   agentnames = ["DiscreteCQL","NFQ","DoubleDQN","DiscreteSAC","DiscreteBCQ","DiscreteBC","DQN"]
   scalers =  [rb,pt,norm,std,mx,noop]
   extractors = [pca,ica,fa,noop]
   dfresults = @sync @distributed (vcat) for agentname in agentnames
      @distributed (vcat) for sc in scalers
         @distributed (vcat) for xt  in extractors
            try
               rlagent = DiscreteRLOffline(agentname,Dict(:runtime_args=>Dict(:n_epochs=>1)))
               rlpipeline = ((numf |> sc |> xt)) |> rlagent 
               res = crossvalidateRL(rlpipeline,observation,action_reward_terminal)
               scn   = sc.name[1:end - 4]; xtn = xt.name[1:end - 4]; lrn = rlagent.name[1:end - 4]
               pname = "$scn |> $xtn |> $lrn"
               if !isnan(res)
                  DataFrame(pipeline=pname,td_error=res)
               else
                  DataFrame()
               end
            catch e
               println("error in $agentname")
               DataFrame()
            end
         end
      end
   end
   #sort!(dfresults,:percent_action_matches,rev=true)
   return dfresults
end
dftable= pipelinesearch()

      From worker 2:	2023-06-21 09:17:39 [debug    ] RoundIterator is selected.
      From worker 2:	2023-06-21 09:17:39 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20230621091739
      From worker 2:	2023-06-21 09:17:39 [debug    ] Fitting scaler...              scaler=min_max
      From worker 2:	2023-06-21 09:17:39 [debug    ] Building models...
      From worker 2:	2023-06-21 09:17:39 [debug    ] Models have been built.
      From worker 2:	2023-06-21 09:17:39 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20230621091739/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory

      From worker 6:	2023-06-21 09:17:39 [debug    ] RoundIterator is selected.
      From worker 6:	2023-06-21 09:17:39 [info     ] Directory is created at d3rlpy_logs/DiscreteBCQ_20230621091739
      From worker 6:	2023-06-21 09:17:39 [debug    ] Fitting scaler...              scaler=min_max
      From worker 7:	2023-06-21 09:17:39 [debug    ] RoundIterator is selected.
      From worker 7:	2023-06-21 09:17:39 [info     ] Directory is created at d3rlpy_logs/DiscreteBC_20230621091739
      From worker 7:	2023-06-21 09:17:39 [debug    ] Fitting scaler...              scaler=min_max
      From worker 6:	2023-06-21 09:17:39 [debug    ] Building models...
      From worker 7:	2023-06-21 09:17:39 [debug    ] Building models...
      From worker 6:	2023-06-21 09:17:39 [debug    ] Models have been built.
      From worker 7:	2023-06-21 09:17:39 [debug    ] Models have been built.
      From worker 6:	2023-06-21 09:17:39 [info     ] Parameters are saved to d3rlpy_logs/DiscreteBCQ_202306210917

      From worker 4:	2023-06-21 09:17:40 [debug    ] RoundIterator is selected.
      From worker 4:	2023-06-21 09:17:40 [info     ] Directory is created at d3rlpy_logs/DoubleDQN_20230621091740
      From worker 4:	2023-06-21 09:17:40 [debug    ] Fitting scaler...              scaler=min_max
      From worker 4:	2023-06-21 09:17:40 [debug    ] Building models...
      From worker 4:	2023-06-21 09:17:40 [debug    ] Models have been built.
      From worker 4:	2023-06-21 09:17:40 [info     ] Parameters are saved to d3rlpy_logs/DoubleDQN_20230621091740/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean'

      From worker 4:	2023-06-21 09:17:41 [info     ] DoubleDQN_20230621091740: epoch=1 step=190 epoch=1 metrics={'time_sample_batch': 4.182489294754831e-05, 'time_algorithm_update': 0.001713250812731291, 'loss': 0.08083449318808944, 'time_step': 0.0017797846543161492, 'td_error': 0.3419192071373942} step=190
      From worker 4:	2023-06-21 09:17:41 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_20230621091740/model_190.pt
      From worker 4:	2023-06-21 09:17:41 [debug    ] RoundIterator is selected.
      From worker 4:	2023-06-21 09:17:41 [info     ] Directory is created at d3rlpy_logs/DoubleDQN_20230621091741
      From worker 4:	2023-06-21 09:17:41 [debug    ] Fitting scaler...              scaler=min_max
      From worker 6:	2023-06-21 09:17:41 [debug    ] RoundIterator is selected.
      From worker 6:	2023-06-21 09:17:41 [info     ] Directory is created at d3rlpy_logs/DiscreteBCQ_20230621091741
      From worker 6:	2023-06-21 09:17:41 [debug    ] Fitting scaler.

      From worker 2:	2023-06-21 09:17:41 [debug    ] Building models...
      From worker 2:	2023-06-21 09:17:41 [debug    ] Models have been built.
      From worker 3:	2023-06-21 09:17:41 [debug    ] RoundIterator is selected.
      From worker 2:	2023-06-21 09:17:41 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20230621091741/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': {'type': 'min_max', 'params': {'maximum': array([[1.7241911, 1.5049787, 1.6107398, 1.5126448, 1.7445312, 0

...

      From worker 3:	2023-06-21 09:18:24 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20230621091824
      From worker 3:	2023-06-21 09:18:24 [debug    ] Fitting scaler...              scaler=min_max
      From worker 3:	2023-06-21 09:18:24 [debug    ] Building models...
      From worker 3:	2023-06-21 09:18:24 [debug    ] Models have been built.
      From worker 3:	2023-06-21 09:18:24 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20230621091824/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'rew

      From worker 5:	2023-06-21 09:18:25 [debug    ] RoundIterator is selected.
Epoch 1/1: 100%|█████████████████| 246/246 [00:00<00:00, 573.03it/s, loss=3.69]
      From worker 3:	2023-06-21 09:18:25 [info     ] DiscreteCQL_20230621091825: epoch=1 step=246 epoch=1 metrics={'time_sample_batch': 2.762554137687373e-05, 'time_algorithm_update': 0.0016954652662199688, 'loss': 3.665842409056377, 'time_step': 0.0017403906922999436, 'td_error': 0.38482568770775694} step=246
      From worker 3:	2023-06-21 09:18:25 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20230621091825/model_246.pt
      From worker 3:	2023-06-21 09:18:25 [debug    ] RoundIterator is selected.
      From worker 5:	2023-06-21 09:18:26 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20230621091826
      From worker 5:	2023-06-21 09:18:26 [debug    ] Fitting scaler...              scaler=min_max
      From worker 5:	2023-06-21 09:18:26 [debug    ] Building models...
      From worker 5:	2023-

Excessive output truncated after 524357 bytes.

      From worker 4:	2023-06-21 09:18:26 [info     ] Directory is created at d3rlpy_logs/DiscreteSAC_20230621091826
      From worker 4:	2023-06-21 09:18:26 [debug    ] Fitting scaler...              scaler=min_max
      From worker 4:	2023-06-21 09:18:27 [debug    ] Building models...
      From worker 4:	2023-06-21 09:18:27 [debug    ] Models have been built.
      From worker 4:	2023-06-21 09:18:27 [info     ] Parameters are saved to d3rlpy_logs/DiscreteSAC_20230621091826/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 0.0001, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 64, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factor

Row,pipeline,td_error
,String,Float64
1,RobustScaler |> PCA |> DiscreteCQL,0.348413
2,RobustScaler |> FastICA |> DiscreteCQL,0.375755
3,PowerTransformer |> PCA |> DiscreteCQL,0.360337
4,PowerTransformer |> FastICA |> DiscreteCQL,0.351133
5,Normalizer |> PCA |> DiscreteCQL,0.358127
6,Normalizer |> FastICA |> DiscreteCQL,0.386176
7,Normalizer |> Noop |> DiscreteCQL,0.37901
8,StandardScaler |> PCA |> DiscreteCQL,0.386691
9,StandardScaler |> Noop |> DiscreteCQL,0.381613


### Results

In [99]:
sort!(dftable,:td_error,rev=false)
dftable

Row,pipeline,td_error
,String,Float64
1,Noop |> FastICA |> DoubleDQN,0.325789
2,StandardScaler |> FastICA |> DQN,0.329664
3,PowerTransformer |> FastICA |> DQN,0.330706
4,PowerTransformer |> FastICA |> DiscreteSAC,0.331318
5,Normalizer |> FastICA |> DQN,0.332194
6,PowerTransformer |> FastICA |> DiscreteBCQ,0.333078
7,RobustScaler |> FastICA |> DQN,0.33313
8,StandardScaler |> Noop |> DoubleDQN,0.33318
9,StandardScaler |> PCA |> DoubleDQN,0.333818


##### Top 5 and last 5 results

In [100]:
first(dftable,5)

Row,pipeline,td_error
,String,Float64
1,Noop |> FastICA |> DoubleDQN,0.325789
2,StandardScaler |> FastICA |> DQN,0.329664
3,PowerTransformer |> FastICA |> DQN,0.330706
4,PowerTransformer |> FastICA |> DiscreteSAC,0.331318
5,Normalizer |> FastICA |> DQN,0.332194


In [101]:
last(dftable,5)

Row,pipeline,td_error
,String,Float64
1,Noop |> FactorAnalysis |> NFQ,2.48772e14
2,PowerTransformer |> Noop |> DQN,5.34366e21
3,PowerTransformer |> Noop |> DoubleDQN,3.26964e22
4,RobustScaler |> PCA |> NFQ,3.56353e25
5,RobustScaler |> PCA |> DiscreteSAC,3.36113e26
